### 1.EDA Analysis

##### 1. Inspecting inactive drivers who has not provided a drive

In [3]:
rides = spark.read.parquet("/mnt/cis442f-data/duocar/clean/rides/")
drivers = spark.read.parquet("/mnt/cis442f-data/duocar/clean/drivers/")
drivers.select('id').subtract(rides.select('driver_id')).count()
# There are 103 inactive drivers.

Out[ 3 ]: 103

##### 2. Plot the ride rating in terms of vehicles

In [5]:
reviews = spark.read.parquet("/mnt/cis442f-data/duocar/clean/ride_reviews/")
joined = rides.select('driver_id','star_rating').join(drivers.select('id','vehicle_make'),rides.driver_id == drivers.id)
eda = joined.groupby('star_rating').pivot('vehicle_make').count()
display(eda)

star_rating,Acura,Audi,BMW,Buick,Cadillac,Chevrolet,Chrysler,Dodge,Ford,GMC,Honda,Hyundai,INFINITI,Jeep,Kia,Lexus,Lincoln,Mazda,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Subaru,Tesla,Toyota,Volkswagen,Volvo
null,135,61,35,33,67,305,119,113,462,138,112,108,74,94,94,60,174,32,133,32,1,72,115,9,262,55,39
1,38,25,20,12,29,101,32,63,164,44,94,58,38,46,73,24,34,16,37,27,null,45,82,7,129,40,8
3,178,116,89,88,135,465,232,222,865,217,309,251,127,209,215,98,195,98,186,77,1,204,287,31,492,137,40
5,959,681,373,310,657,2940,1069,1048,4711,1289,1011,1206,710,1044,897,572,1616,390,1102,271,6,936,1185,111,2429,631,231
4,193,135,106,79,166,667,291,320,1170,270,338,322,171,292,270,113,300,103,259,105,2,253,332,47,640,199,52
2,93,60,46,31,86,236,131,138,438,101,219,147,67,149,148,54,114,57,79,43,4,111,158,7,322,94,22
0,8,6,4,3,4,13,9,17,39,13,22,7,3,11,18,2,4,5,11,5,1,10,16,1,21,2,1


In [6]:
display(eda)

star_rating,Acura,Audi,BMW,Buick,Cadillac,Chevrolet,Chrysler,Dodge,Ford,GMC,Honda,Hyundai,INFINITI,Jeep,Kia,Lexus,Lincoln,Mazda,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Subaru,Tesla,Toyota,Volkswagen,Volvo
null,135,61,35,33,67,305,119,113,462,138,112,108,74,94,94,60,174,32,133,32,1,72,115,9,262,55,39
1,38,25,20,12,29,101,32,63,164,44,94,58,38,46,73,24,34,16,37,27,null,45,82,7,129,40,8
3,178,116,89,88,135,465,232,222,865,217,309,251,127,209,215,98,195,98,186,77,1,204,287,31,492,137,40
5,959,681,373,310,657,2940,1069,1048,4711,1289,1011,1206,710,1044,897,572,1616,390,1102,271,6,936,1185,111,2429,631,231
4,193,135,106,79,166,667,291,320,1170,270,338,322,171,292,270,113,300,103,259,105,2,253,332,47,640,199,52
2,93,60,46,31,86,236,131,138,438,101,219,147,67,149,148,54,114,57,79,43,4,111,158,7,322,94,22
0,8,6,4,3,4,13,9,17,39,13,22,7,3,11,18,2,4,5,11,5,1,10,16,1,21,2,1


From the barplot and line chart, star rating is not dependent on vehicle make. The bar plot and line chart both indicates that 5 stars is the most frequently given rating. And the distribution of rating is similar regardless of vehicle make. Therefore, we can conclude that star rating is not dependent on vehicle make.

##### 3. Investigate student users

In [9]:
# create new columns of the time information (date_time in a day, day of a week, month)
riders = spark.read.parquet("/mnt/cis442f-data/duocar/clean/riders/")
joined2 = rides.join(riders,rides.rider_id == riders.id).select('rider_id','student','sex','date_time')

from pyspark.sql.functions import dayofweek,month,hour,when
joined2 = joined2.select('rider_id','student','sex',\
                         'date_time',dayofweek('date_time').alias('dayofweek'),month('date_time').alias('month'))\
          .withColumn('date_time', when((hour(joined2.date_time) > 5) & (hour(joined2.date_time) < 13),'monrning')\
                  .when((hour(joined2.date_time) > 12) & (hour(joined2.date_time) < 19),'afternoon').\
                   when((hour(joined2.date_time) > 18) & (hour(joined2.date_time) < 24),'evening').otherwise('midnight'))
joined2.show(5)


+------------+-------+------+---------+---------+-----+
 rider_id|student| sex|date_time|dayofweek|month|
+------------+-------+------+---------+---------+-----+
220200000084| false|female| monrning| 4| 2|
220200000462| false| male| monrning| 4| 2|
220200000489| false| male| monrning| 4| 2|
220200000057| false| male| monrning| 4| 2|
220200000012| true| null| monrning| 4| 2|
+------------+-------+------+---------+---------+-----+
only showing top 5 rows

In [10]:
# 1. time period during a day
datetime = joined2.dropna(subset = ['sex']).filter(joined2.student == True).groupby('sex').pivot('date_time').count()
display(datetime)


sex,afternoon,evening,midnight,monrning
female,641,930,786,218
male,742,1073,887,233


The bar chart indicates that student prefer taking a ride during evening and midnight (7pm to 5am the next day). Student seldom take a ride in the morning. Also, the distribution has no sex difference.

In [12]:
# 2.Month
month = joined2.dropna(subset = ['sex']).filter(joined2.student == True).groupby('sex').pivot('month').count()
display(month)

sex,2,3,4
female,569,966,1040
male,643,1179,1113


The bar chart indicates that females prefer taking a ride in April to February and March, while males prefer March. Both genders tend to take less rides in February.

In [14]:
# 3. day of week
dayofweek = joined2.dropna(subset = ['sex']).filter(joined2.student == True).groupby('sex').pivot('dayofweek').count()
display(dayofweek)


sex,1,2,3,4,5,6,7
female,402,316,382,362,320,390,403
male,404,378,428,371,438,464,452


The bar chart shows that males tend to take more rides than females. Males prefer to take a ride during weekends and Fridays. And females take the most rides in Mondays and Sundays.

##### 4. Explore the distance data

In [17]:
rides.where(rides.distance.isNull()).select('cancelled','distance').distinct().show()

+---------+--------+
cancelled|distance|
+---------+--------+
 true| null|
+---------+--------+

All rides records that has nulls in distance is due to cancelled trips.